In [1]:
import json
import os
import pandas as pd

In [2]:
os.chdir('../')

## Connect to the server

In [3]:
filename_credentials = 'clickhouse_credentials.json'

In [4]:
with open(filename_credentials, 'r') as f:
    config = json.load(f)

In [5]:
config

{'host': '35.210.233.219',
 'port': 9001,
 'user': 'user1',
 'password': 'hola1234',
 'database': 'default'}

In [6]:
from src.data.load_data import connect_clickhouse

In [7]:
client = connect_clickhouse(config)

In [8]:
client.execute('SHOW TABLES')

[]

## Create a schema

In [9]:
database = 'raw'

In [10]:
client.execute('CREATE DATABASE IF NOT EXISTS ' + database)

[]

In [11]:
client.execute('SHOW DATABASES')

[('_temporary_and_external_tables',), ('default',), ('raw',), ('system',)]

## Load data

In [12]:
schema = {
    'meal_id' : int,
    'category' : str,
    'cuisine' : str
}

In [13]:
from src.data.load_data import load_table_from_csv

In [14]:
table_name = 'meal'
file_path = '../api/data/raw/meal_info.csv'

In [17]:
client.execute('''
    CREATE TABLE IF NOT EXISTS ''' + database + '''. ''' + table_name + '''
    (
        meal_id Int32,
        category String,
        cuisine String
    )
    ENGINE = Log
''')

[]

In [16]:
client.execute('SHOW TABLES')

[('test',)]

In [18]:
load_table_from_csv(client, table_name, file_path, schema, database)

In [16]:
data, columns = client.execute("SELECT * FROM test", with_column_types=True)

In [18]:
columns

[('meal_id', 'Int32'), ('category', 'String'), ('cuisine', 'String')]

In [20]:
df = pd.DataFrame(data, columns = [i[0] for i in columns])

In [21]:
df.head()

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian


In [22]:
Client.insert_dataframe

ModuleNotFoundError: No module named 'clickhouse_driver.Client'

In [23]:
!pip install clickhouse-driver --upgrade

Requirement already up-to-date: clickhouse-driver in c:\users\smarti25\appdata\local\continuum\anaconda3\lib\site-packages (0.1.5)


In [24]:
!pip install git+https://github.com/mymarilyn/clickhouse-driver@master#egg=clickhouse-driver

In [31]:
client.execute('SELECT category, COUNT(DISTINCT meal_id) AS nb_meals FROM test GROUP BY category ORDER BY nb_meals DESC')

[('Beverages', 12),
 ('Pizza', 3),
 ('Desert', 3),
 ('Pasta', 3),
 ('Salad', 3),
 ('Extras', 3),
 ('Soup', 3),
 ('Starters', 3),
 ('Sandwich', 3),
 ('Seafood', 3),
 ('Fish', 3),
 ('Biryani', 3),
 ('Other Snacks', 3),
 ('Rice Bowl', 3)]